In [ ]:
!pip install torch torchvision ultralytics albumentations scikit-learn opencv-python pillow matplotlib

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("tapakah68/supervisely-filtered-segmentation-person-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/supervisely-filtered-segmentation-person-dataset


In [ ]:
import shutil

# Copia todo el dataset a /content/fashionpedia
#shutil.copytree(path, '/content/fashionpedia', dirs_exist_ok=True)
shutil.copytree(path, '/content/persons', dirs_exist_ok=True)


'/content/persons'

# **Sistema de Segmentación de Personas y Extracción de Colores con Transfer Learning**

Este proyecto implementa un sistema completo que:
- Segmenta personas en imágenes usando YOLOv8 con máscaras.
- Extrae colores dominantes del torso y piernas de las personas detectadas.
- Aplica Transfer Learning para adaptar un modelo preentrenado a un nuevo dataset.
- Genera visualizaciones y resultados en formato JSON.

### **Importación de Bibliotecas**

En esta sección importamos todas las dependencias necesarias para el proyecto:

- **PyTorch**: Framework principal para el manejo de redes neuronales
- **OpenCV (cv2)**: Procesamiento de imágenes y visión por computadora
- **YOLO (Ultralytics)**: Implementación del modelo de segmentación
- **Scikit-learn**: Para algoritmos de clustering (K-means)
- **Otras utilidades**: Manipulación de archivos, visualización, etc.

Estas bibliotecas nos permitirán desde cargar imágenes hasta entrenar modelos de deep learning.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from ultralytics import YOLO
import cv2
import numpy as np
import os
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans
from collections import Counter
import yaml
from pathlib import Path
import json
import pandas as pd


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


### **Configuración de Paleta de Colores**

Definimos una paleta de 18 colores básicos en español que usaremos para clasificar las prendas de vestir. Cada color tiene:

- Nombre en español (ej. "azul_marino")
- Valores RGB de referencia
- Se usarán para etiquetar automáticamente los colores detectados en las prendas

Esta configuración es especialmente útil para aplicaciones de moda y retail donde los nombres de colores deben ser comprensibles para usuarios finales.

In [ ]:
COLORS = [
   'negro', 'blanco', 'gris', 'azul', 'azul_marino', 'azul_claro',
   'rojo', 'rosa', 'verde', 'verde_oscuro', 'amarillo', 'naranja',
   'marron', 'beige', 'violeta', 'morado', 'dorado', 'plateado'
]

### **Dataset Supervisely para Segmentación**

La clase `SuperviselyDataset` maneja el conjunto de datos [Human Segmentation Dataset](https://supervise.ly/) que contiene:

- **Imágenes**: Fotografías de personas con diversas vestimentas
- **Máscaras**: Anotaciones casi pixel-perfect de las siluetas humanas

Funcionalidades:
- Carga automática de pares imagen/máscara
- Filtrado de imágenes corruptas o sin anotaciones
- Compatible con el formato YOLOv8 para segmentación


In [ ]:
class SuperviselyDataset(Dataset):
    """Dataset para entrenar YOLO con el dataset Human Segmentation Dataset - Supervise.ly"""
    def __init__(self, images_dir, masks_dir, img_size=640):
        self.images_dir = Path(images_dir)
        self.masks_dir = Path(masks_dir)
        self.img_size = img_size

        # Obtener lista de imágenes válidas
        self.valid_samples = []
        for img_path in self.images_dir.glob('*.png'):
            # Buscar máscara correspondiente (puede ser .png o .jpg)
            mask_path_png = self.masks_dir / f"{img_path.stem}.png"
            mask_path_jpg = self.masks_dir / f"{img_path.stem}.jpg"

            if mask_path_png.exists():
                self.valid_samples.append((img_path, mask_path_png))
            elif mask_path_jpg.exists():
                self.valid_samples.append((img_path, mask_path_jpg))

        print(f"Encontradas {len(self.valid_samples)} imágenes válidas con máscaras")

    def __len__(self):
        return len(self.valid_samples)

    def __getitem__(self, idx):
        img_path, mask_path = self.valid_samples[idx]
        return str(img_path), str(mask_path)

### **Extracción de Colores Dominantes**

El `ColorExtractor` realiza:

1. **Clustering con K-means**: Identifica los colores predominantes en una región
2. **Filtrado**:
   - Elimina píxeles demasiado oscuros/claros (sombras/reflejos)
   - Manejo de casos especiales (regiones vacías)
3. **Mapeo a nombres**:
   - Convierte valores RGB a nombres comprensibles
   - Usa distancias euclidianas para encontrar el color más cercano

Ejemplo de uso:
```python
extractor = ColorExtractor()
color_name, rgb = extractor.extract_dominant_color(region_imagen)

In [ ]:
class ColorExtractor:
    """Extractor de colores dominantes mejorado"""
    def __init__(self):
        self.color_names = COLORS

    def extract_dominant_color(self, image_region, n_colors=3):
        """Extrae el color dominante de una región"""
        if image_region.size == 0:
            return 'gris', [128, 128, 128]

        # Reshape para K-means
        pixels = image_region.reshape(-1, 3)

        # Remover píxeles muy oscuros o muy claros (posibles sombras/reflejos)
        pixels = pixels[np.all(pixels > 20, axis=1)]  # Remover muy oscuros
        pixels = pixels[np.all(pixels < 235, axis=1)]  # Remover muy claros

        if len(pixels) < 10:
            return 'gris', [128, 128, 128]

        # Aplicar K-means
        kmeans = KMeans(n_clusters=min(n_colors, len(pixels)), random_state=42, n_init=10)
        kmeans.fit(pixels)

        # Obtener colores y sus frecuencias
        colors = kmeans.cluster_centers_
        labels = kmeans.labels_

        # Contar frecuencias
        label_counts = Counter(labels)

        # Obtener el color más frecuente
        dominant_idx = label_counts.most_common(1)[0][0]
        dominant_color_rgb = colors[dominant_idx].astype(int)

        # Convertir RGB a nombre de color
        color_name = self.rgb_to_color_name(dominant_color_rgb)

        return color_name, dominant_color_rgb

    def rgb_to_color_name(self, rgb):
        """Convierte RGB a nombre de color más preciso"""
        color_map = {
            'negro': [0, 0, 0],
            'blanco': [255, 255, 255],
            'gris': [128, 128, 128],
            'azul': [0, 0, 255],
            'azul_marino': [0, 0, 128],
            'azul_claro': [173, 216, 230],
            'rojo': [255, 0, 0],
            'rosa': [255, 192, 203],
            'verde': [0, 255, 0],
            'verde_oscuro': [0, 100, 0],
            'amarillo': [255, 255, 0],
            'naranja': [255, 165, 0],
            'marron': [165, 42, 42],
            'beige': [245, 245, 220],
            'violeta': [238, 130, 238],
            'morado': [128, 0, 128],
            'dorado': [255, 215, 0],
            'plateado': [192, 192, 192]
        }

        min_distance = float('inf')
        closest_color_name = 'gris'

        for color_name, color_rgb in color_map.items():
            distance = np.sqrt(sum((c1 - c2) ** 2 for c1, c2 in zip(rgb, color_rgb)))
            if distance < min_distance:
                min_distance = distance
                closest_color_name = color_name

        return closest_color_name


## **Clase PersonSegmentationSystem**
### **Sistema de Segmentación de Personas con Transfer Learning**

Clase central que implementa un pipeline completo para:
1. Segmentación precisa de personas usando YOLOv8
2. Análisis de color de prendas de vestir
3. Generación de reportes visuales y JSON

**Arquitectura basada en Transfer Learning**:
- Utiliza YOLOv8-seg como modelo base (pre-entrenado en COCO)
- Fine-tuning especializado en segmentación de personas
- Capas convolucionales congeladas en primeras etapas
- Head de segmentación ajustable

### **Métodos de Procesamiento**
1. `prepare_supervisely_dataset()`:
   - Convierte un dataset Supervise.ly al formato YOLOv8-seg
   - Extrae contornos de máscaras binarizadas
   - Genera etiquetas .txt con coordenadas normalizadas
   - Organiza carpetas de imágenes y etiquetas (train/val)
   - Crea archivo YAML con configuración del dataset

2. `train_yolo_person_segmentation()`:
   - Realiza el fine-tuning del modelo YOLOv8
   - Ajusta pesos del modelo base al nuevo dominio
   - Imprime métricas por época si están disponibles

3. `load_trained_model()`:
   - Carga de un modelo YOLOv8-seg previamente entrenado
   - Permite realizar inferencia sin repetir el entrenamiento

4. `segment_and_extract_colors()`:
   - Segmentación con confianza ajustable
   - División inteligente en regiones (torso/piernas)
   - Extracción de color dominante por región

5. `visualize_segmentation_results()`:
   - Visualización comparativa
   - Overlay de colores semitransparente
   - Exportación a alta resolución

6. `batch_process_images()`:
   - Procesamiento paralelizado
   - Generación de reportes JSON
   - Resumen estadístico


In [ ]:
class PersonSegmentationSystem:
    """Sistema completo: Segmentación de personas + extracción de colores"""

    def __init__(self, base_model='yolov8n-seg.pt'):
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        print(f"Usando dispositivo: {self.device}")

        # Modelo YOLO de segmentación
        self.yolo_model = YOLO(base_model)
        self.color_extractor = ColorExtractor()

        #print(yolo_model.names) para verificar las clases si se desea

        # Configuración para crear dataset YOLO
        self.yolo_config = {
            'train': '',
            'val': '',
            'nc': 1,  # Solo clase persona
            'names': {0: 'person'}
        }

    def prepare_supervisely_dataset(self, images_dir, masks_dir, output_dir):
        """Convierte dataset Supervise.ly para entrenar YOLO en segmentación de personas"""

        # Crear directorios de salida
        output_path = Path(output_dir)
        (output_path / 'images' / 'train').mkdir(parents=True, exist_ok=True)
        (output_path / 'images' / 'val').mkdir(parents=True, exist_ok=True)
        (output_path / 'labels' / 'train').mkdir(parents=True, exist_ok=True)
        (output_path / 'labels' / 'val').mkdir(parents=True, exist_ok=True)

        # Procesar imágenes
        images_dir = Path(images_dir)
        masks_dir = Path(masks_dir)

        processed_count = 0

        for img_path in images_dir.glob('*.png'):
            # Buscar máscara correspondiente
            mask_path_png = masks_dir / f"{img_path.stem}.png"
            mask_path_jpg = masks_dir / f"{img_path.stem}.jpg"

            mask_path = None
            if mask_path_png.exists():
                mask_path = mask_path_png
            elif mask_path_jpg.exists():
                mask_path = mask_path_jpg

            if mask_path is None:
                continue

            # Cargar imagen y máscara
            image = cv2.imread(str(img_path))
            mask = cv2.imread(str(mask_path), cv2.IMREAD_GRAYSCALE)

            if image is None or mask is None:
                continue

            h, w = image.shape[:2]

            # Binarizar máscara
            _, mask_binary = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)

            # Encontrar contornos de personas
            contours, _ = cv2.findContours(mask_binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            if len(contours) == 0:
                continue

            # Determinar si va a train o val (80-20 split)
            subset = 'train' if processed_count % 5 != 0 else 'val'

            # Copiar imagen
            dst_img = output_path / 'images' / subset / img_path.name
            cv2.imwrite(str(dst_img), image)

            # Crear etiquetas YOLO para segmentación
            dst_label = output_path / 'labels' / subset / f"{img_path.stem}.txt"

            with open(dst_label, 'w') as f:
                for contour in contours:
                    # Obtener bbox del contorno
                    x, y, w_bbox, h_bbox = cv2.boundingRect(contour)

                    # Filtrar contornos muy pequeños
                    if w_bbox < 20 or h_bbox < 20:
                        continue

                    # Convertir contorno a coordenadas normalizadas
                    contour_normalized = []
                    for point in contour:
                        x_norm = point[0][0] / w
                        y_norm = point[0][1] / h
                        contour_normalized.extend([x_norm, y_norm])

                    # Simplificar contorno si es muy complejo
                    if len(contour_normalized) > 200:  # Máximo 100 puntos
                        epsilon = 0.01 * cv2.arcLength(contour, True)
                        contour_simplified = cv2.approxPolyDP(contour, epsilon, True)
                        contour_normalized = []
                        for point in contour_simplified:
                            x_norm = point[0][0] / w
                            y_norm = point[0][1] / h
                            contour_normalized.extend([x_norm, y_norm])

                    # Escribir en formato YOLO segmentación
                    if len(contour_normalized) >= 6:  # Mínimo 3 puntos
                        line = "0 " + " ".join(map(str, contour_normalized))
                        f.write(line + "\n")

            processed_count += 1

            if processed_count % 100 == 0:
                print(f"Procesadas {processed_count} imágenes...")

        # Crear archivo de configuración YOLO
        self.yolo_config['train'] = str(output_path / 'images' / 'train')
        self.yolo_config['val'] = str(output_path / 'images' / 'val')

        config_path = output_path / 'person_segmentation.yaml'
        with open(config_path, 'w') as f:
            yaml.dump(self.yolo_config, f)

        print(f"Dataset preparado: {processed_count} imágenes procesadas")
        print(f"Configuración guardada en: {config_path}")

        return str(config_path)

    def train_yolo_person_segmentation(self, dataset_config_path, epochs=100, batch_size=16):
        """Entrena YOLO para segmentación de personas"""

        print("Iniciando entrenamiento YOLO para segmentación de personas...")

        # Entrenar modelo
        results = self.yolo_model.train(
            data=dataset_config_path,
            epochs=epochs,
            batch=batch_size,
            imgsz=640,
            device=self.device,
            workers=4,
            patience=10,
            save=True,
            project='person_segmentation',
            name='yolo_person_seg_v1'
        )

        print("Entrenamiento completado!")

        # Mostrar loss y métricas por época
        results_csv = "person_segmentation/yolo_person_seg_v1/results.csv"
        if os.path.exists(results_csv):
            df = pd.read_csv(results_csv)
            print("\nResumen de entrenamiento (loss y métricas por época):")
            print(df[["epoch", "train/box_loss", "train/seg_loss", "metrics/mAP_0.5"]])
        else:
            print("No se encontró el archivo de resultados para mostrar métricas.")

        return results

    def load_trained_model(self, model_path):
        """Cargar modelo YOLO entrenado"""
        self.yolo_model = YOLO(model_path)
        print(f"Modelo cargado: {model_path}")

    def segment_and_extract_colors(self, image_path, conf_threshold=0.5):
        """Segmenta personas y extrae colores de torso y piernas usando las máscaras"""

        # Realizar segmentación
        results = self.yolo_model(image_path, conf=conf_threshold)

        # Cargar imagen original
        image = cv2.imread(image_path)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        h, w = image_rgb.shape[:2]

        detections = []

        for result in results:
            if result.masks is not None:
                boxes = result.boxes
                masks = result.masks

                for i in range(len(boxes)):
                    # Obtener bbox y confianza
                    x1, y1, x2, y2 = boxes.xyxy[i].cpu().numpy()
                    confidence = boxes.conf[i].cpu().numpy()

                    # Obtener máscara
                    mask = masks.data[i].cpu().numpy()

                    # Redimensionar máscara al tamaño original
                    mask_resized = cv2.resize(mask, (w, h), interpolation=cv2.INTER_NEAREST)

                    # Crear máscara binaria
                    mask_binary = (mask_resized > 0.5).astype(np.uint8)

                    # Aplicar máscara a la imagen
                    person_pixels = image_rgb[mask_binary == 1]

                    if len(person_pixels) > 0:
                        # Dividir la máscara en regiones de torso y piernas
                        mask_coords = np.where(mask_binary == 1)
                        y_coords = mask_coords[0]

                        # Calcular límites para torso y piernas
                        y_min, y_max = np.min(y_coords), np.max(y_coords)
                        person_height = y_max - y_min

                        # Torso: 60% superior
                        torso_limit = y_min + int(person_height * 0.6)

                        # Crear máscaras para torso y piernas
                        torso_mask = mask_binary.copy()
                        torso_mask[torso_limit:, :] = 0

                        legs_mask = mask_binary.copy()
                        legs_mask[:torso_limit, :] = 0

                        # Extraer píxeles de torso y piernas
                        torso_pixels = image_rgb[torso_mask == 1]
                        legs_pixels = image_rgb[legs_mask == 1]

                        # Extraer colores dominantes
                        torso_color, torso_rgb = self.color_extractor.extract_dominant_color(torso_pixels)
                        legs_color, legs_rgb = self.color_extractor.extract_dominant_color(legs_pixels)

                        detections.append({
                            'bbox': [int(x1), int(y1), int(x2-x1), int(y2-y1)],
                            'confidence': float(confidence),
                            'mask': mask_binary,
                            'torso_color': torso_color,
                            'torso_rgb': torso_rgb.tolist(),
                            'legs_color': legs_color,
                            'legs_rgb': legs_rgb.tolist()
                        })

        return detections

    def visualize_segmentation_results(self, image_path, detections, save_path=None):
        """Visualiza resultados de segmentación y colores"""

        image = cv2.imread(image_path)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        fig, axes = plt.subplots(1, 2, figsize=(20, 10))

        # Imagen original con bboxes
        axes[0].imshow(image_rgb)
        axes[0].set_title('Detección Original', fontsize=14)

        # Imagen con máscaras
        axes[1].imshow(image_rgb)
        axes[1].set_title('Segmentación con Colores', fontsize=14)

        colors_overlay = np.zeros_like(image_rgb)

        for i, detection in enumerate(detections):
            bbox = detection['bbox']
            x, y, w, h = bbox
            mask = detection['mask']

            # Dibujar bbox en imagen original
            rect = plt.Rectangle((x, y), w, h, fill=False, color='red', linewidth=2)
            axes[0].add_patch(rect)

            # Crear overlay de colores para la segmentación
            torso_rgb = np.array(detection['torso_rgb'])
            legs_rgb = np.array(detection['legs_rgb'])

            # Dividir máscara en torso y piernas
            mask_coords = np.where(mask == 1)
            y_coords = mask_coords[0]

            if len(y_coords) > 0:
                y_min, y_max = np.min(y_coords), np.max(y_coords)
                person_height = y_max - y_min
                torso_limit = y_min + int(person_height * 0.6)

                # Colorear torso
                torso_region = mask.copy()
                torso_region[torso_limit:, :] = 0
                colors_overlay[torso_region == 1] = torso_rgb * 0.6

                # Colorear piernas
                legs_region = mask.copy()
                legs_region[:torso_limit, :] = 0
                colors_overlay[legs_region == 1] = legs_rgb * 0.6

            # Información de colores
            confidence = detection['confidence']
            torso_color = detection['torso_color']
            legs_color = detection['legs_color']

            text = f"Persona {i+1}\nTorso: {torso_color}\nPiernas: {legs_color}\nConf: {confidence:.2f}"

            axes[0].text(x, y-60, text, fontsize=10, color='white', weight='bold',
                        bbox=dict(boxstyle="round,pad=0.5", facecolor="red", alpha=0.8))

        # Superponer colores
        axes[1].imshow(colors_overlay, alpha=0.7)

        for ax in axes:
            ax.axis('off')

        plt.tight_layout()

        if save_path:
            plt.savefig(save_path, bbox_inches='tight', dpi=300)
            print(f"Resultado guardado: {save_path}")

        plt.show()

    def batch_process_images(self, images_dir, output_dir):
        """Procesa múltiples imágenes y guarda resultados"""

        images_dir = Path(images_dir)
        output_dir = Path(output_dir)
        output_dir.mkdir(exist_ok=True)

        results_summary = []

        for img_path in images_dir.glob('*.png'):
            print(f"Procesando: {img_path.name}")

            # Segmentar y extraer colores
            detections = self.segment_and_extract_colors(str(img_path))

            # Guardar visualización
            save_path = output_dir / f"segmentation_result_{img_path.name}"
            self.visualize_segmentation_results(str(img_path), detections, str(save_path))

            # Preparar datos para JSON (sin la máscara binaria)
            json_detections = []
            for det in detections:
                json_det = det.copy()
                json_det.pop('mask', None)  # Remover máscara para JSON
                json_detections.append(json_det)

            # Guardar datos en JSON
            result_data = {
                'image': img_path.name,
                'detections': json_detections,
                'total_persons': len(detections)
            }

            results_summary.append(result_data)

            # Mostrar resumen
            print(f"  Detectadas {len(detections)} personas")
            for i, det in enumerate(detections):
                print(f"    Persona {i+1}: Torso={det['torso_color']}, "
                      f"Piernas={det['legs_color']}, Conf={det['confidence']:.2f}")

        # Guardar resumen
        with open(output_dir / 'segmentation_results_summary.json', 'w') as f:
            json.dump(results_summary, f, indent=2)

        print(f"\nProcesamiento completado. Resultados en: {output_dir}")
        return results_summary

### **Demo: Flujo de Trabajo Completo**

La función `main()` muestra un ejemplo completo con 3 pasos:

1. **Preparación de datos**:
   - Conversión del formato Supervisely a YOLO
   - División train/val (80/20)

2. **Entrenamiento (Transfer Learning):**
   - Inicio del fine-tuning del modelo YOLOv8-seg
   - Reutiliza pesos preentrenados (COCO) y los ajusta al nuevo dataset
   - Monitoreo de métricas por época (pérdida, mAP)

3. **Inferencia**:
   - Procesamiento de imágenes nuevas
   - Generación de reportes visuales

In [ ]:
def main():
    """Función principal de demostración"""

    # Configurar rutas para el dataset Human Segmentation Dataset - Supervise.ly
    SUPERVISELY_ROOT = "/content/persons/supervisely_person_clean_2667_img/supervisely_person_clean_2667_img"
    SUPERVISELY_IMAGES = f"{SUPERVISELY_ROOT}/images"
    SUPERVISELY_MASKS = f"{SUPERVISELY_ROOT}/masks"

    PERSON_DATASET_DIR = "/content/persons_segmented"
    TEST_IMAGES_DIR = "/content/test_images"
    OUTPUT_DIR = "/content/results"

    # Crear sistema
    system = PersonSegmentationSystem()

    # Paso 1: Preparar dataset de segmentación desde Supervise.ly
    print("=== PASO 1: Preparando dataset de segmentación ===")
    dataset_config = system.prepare_supervisely_dataset(
        SUPERVISELY_IMAGES,
        SUPERVISELY_MASKS,
        PERSON_DATASET_DIR
    )

    # Paso 2: Entrenar YOLO para segmentación de personas
    print("\n=== PASO 2: Entrenando YOLO para segmentación ===")
    system.train_yolo_person_segmentation(
        dataset_config,
        epochs=10,
        batch_size=8
    )

    #print("\n=== PROCESO COMPLETADO ===")
    #print(f"Resultados guardados en: {OUTPUT_DIR}")

if __name__ == "__main__":
    main()

Usando dispositivo: cuda


100%|██████████| 6.74M/6.74M [00:00<00:00, 153MB/s]


=== PASO 1: Preparando dataset de segmentación ===
Procesadas 100 imágenes...
Procesadas 200 imágenes...
Procesadas 300 imágenes...
Procesadas 400 imágenes...


KeyboardInterrupt: 

In [ ]:
   # Paso 3: Cargar modelo entrenado
   print("\n=== PASO 3: Cargando modelo entrenado ===")
   model_path = "person_segmentation/yolo_person_seg_v1/weights/best.pt"
   system.load_trained_model(model_path)

   # Paso 4: Procesar imágenes de test
   print("\n=== PASO 4: Procesando imágenes de test ===")
   results = system.batch_process_images(TEST_IMAGES_DIR, OUTPUT_DIR)

   print("\n=== PROCESO COMPLETADO ===")
   print(f"Resultados guardados en: {OUTPUT_DIR}")

In [ ]:
# Ajusta la ruta al modelo entrenado
model_path = "/content/person_segmentation/yolo_person_seg_v13/weights/best.pt"

# Define las rutas si no están ya definidas
TEST_IMAGES_DIR = "/content/test_images"  # Asegúrate de que esta ruta sea correcta
OUTPUT_DIR = "/content/results"      # Asegúrate de que esta ruta sea correcta

# Crear una instancia del sistema si no existe (esto puede ser redundante si se llama después de main)
# Si ya tienes la instancia 'system' creada y las rutas definidas:
# system.load_trained_model(model_path)

# Crear sistema
system = PersonSegmentationSystem()

system.load_trained_model(model_path)

# Procesa las imágenes de test y guarda los resultados
results = system.batch_process_images(TEST_IMAGES_DIR, OUTPUT_DIR)

print("\n=== PROCESO COMPLETADO ===")
print(f"Resultados guardados en: {OUTPUT_DIR}")

**OLD VERSION**

In [ ]:

# Rutas (ajusta si es necesario)
MODEL_PATH = "person_detection/yolo_person_v1/weights/best.pt"
TEST_IMAGES_DIR = "/content/test_images"
OUTPUT_DIR = "/content/results"

# Crear sistema y cargar modelo entrenado
system = PersonColorDetectionSystem()
system.load_trained_model(MODEL_PATH)

# Procesar imágenes de test (no reentrena nada)
results = system.batch_process_images(TEST_IMAGES_DIR, OUTPUT_DIR)

print("Resultados guardados en:", OUTPUT_DIR)

In [ ]:
!ls


In [ ]:
!du -sh person_detection


In [ ]:
!rm -r test_images/